In [1]:
from collections import deque

class Graph:
    def __init__(self,adjac_lis):
        self.adjac_lis=adjac_lis
    def get_neighbors(self,v):
        return self.adjac_lis[v]
    
    def h(self,n):
        H={
            'A':1,
            'B':1,
            'C':1,
            'D':1
        }
        
        return H[n]
    def a_star_algorithm(self,start,stop):
        open_lst=set([start])
        closed_lst=set([])
        
        poo={}
        poo[start]=0
        
        par={}
        par[start]=start
        
        while len(open_lst) >0 :
            n=None
            
            for v in open_lst:
                if n==None or poo[v]+self.h(v) < poo[n]+self.h(n):
                    n=v
            if n==None:
                print('Path does not exist')
                return None
            
            if n==stop:
                reconst_path=[]
                
                while par[n]!=n:
                    reconst_path.append(n)
                    n=par[n]
                reconst_path.append(start)
                reconst_path.reverse()
                
                print('Path found : {}'.format(reconst_path))
                return reconst_path
            for (m,weight) in self.get_neighbors(n):
                if m not in open_lst and m not in closed_lst:
                    open_lst.add(m)
                    par[m]=n
                    
                    poo[m]=poo[n]+weight
                else:
                    if poo[m]>poo[n]+weight:
                        poo[m]=poo[n]+weight
                        par[m]=n
                        
                        if m in closed_lst:
                            closed_lst.remove(m)
                            open_lst.add(m)
            
            open_lst.remove(n)
            closed_lst.add(n)
        print('Path does not exist!')
        return None
                        
                    

adjac_lis={
    'A':[('B',1),('C',3),('D',7)],
    'B':[('D',5)],
    'C':[('D',12)]
}

graph1=Graph(adjac_lis)
graph1.a_star_algorithm('A','D')

Path found : ['A', 'B', 'D']


['A', 'B', 'D']

In [3]:
import pandas as pd
import math
import numpy as np

data=pd.read_csv('play.csv')
features=[feat for feat in data]
features.remove("classification")

class Node:
    def __init__(self):
        self.children=[]
        self.value=""
        self.pred=""
        self.isLeaf=False
def entropy(examples):
    pos=0.0
    neg=0.0
    for _, row in examples.iterrows():
        if row["classification"] == "Yes":
            pos += 1
        else:
            neg += 1
    if pos == 0.0 or neg == 0.0:
        return 0.0
    else:
        p = pos / (pos + neg)
        n = neg / (pos + neg)
        return -(p * math.log(p, 2) + n * math.log(n, 2))
def info_gain(examples,attr):
    uniq=np.unique(examples[attr])
    gain=entropy(examples)
    
    for u in uniq:
        subdata=examples[examples[attr]==u]
        sub_e=entropy(subdata)
        
        gain -=(float(len(subdata))/float(len(examples)))*sub_e
        
    return gain
def ID3(examples,attrs):
    root=Node()
    
    max_gain=0
    max_feat=""
    
    for feature in attrs:
        gain=info_gain(examples,feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    root.value=max_feat
    uniq = np.unique(examples[max_feat])
    #print ("\n",uniq)
    for u in uniq:
        #print ("\n",u)
        subdata = examples[examples[max_feat] == u]
        #print ("\n",subdata)
        if entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["classification"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            child = ID3(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)
    return root
        
def printTree(root:Node,depth=0):
    for i in range(depth):
        print("\t",end="")
    print(root.value,end="")
    if root.isLeaf:
        print("->",root.pred)
    print()
    for child in root.children:
        printTree(child,depth+1)
root = ID3(data, features)
printTree(root)


A3
	High
		A1
			False
				A2
					Cool-> ['No']

					Hot-> ['Yes']

			True-> ['No']

	Normal-> ['Yes']



In [4]:
import numpy as np
X=np.array(([2,9],[1,5],[3,6]),dtype=float)
y=np.array(([92],[86],[89]),dtype=float)
X=X/np.amax(X,axis=0)
y=y/100


def sigmoid(x):
    return (1/(1+np.exp(-x)))
def derivative_sigmoid(x):
    return x*(1-x)

epoch=7000
lr=0.1
inputlayer_neuron=2
hiddenlayer_nueron=3
output_neuron=1

wh=np.random.uniform(size=(inputlayer_neuron,hiddenlayer_nueron))
bh=np.random.uniform(size=(1,hiddenlayer_nueron))
wout=np.random.uniform(size=(hiddenlayer_nueron,output_neuron))
bout=np.random.uniform(size=(1,output_neuron))

for i in range(epoch):
    hinp1=np.dot(X,wh)
    hinp=hinp1+bh
    hlayer_act=sigmoid(hinp)
    outinp1=np.dot(hlayer_act,wout)
    outinp = outinp1+bout
    output =sigmoid(outinp)
    
    
    EO=y-output
    outgrad=derivative_sigmoid(output)
    d_output=EO*outgrad
    
    EH=d_output.dot(wout.T)
    hiddengrad=derivative_sigmoid(hlayer_act)
    d_hiddenlayer=EH*hiddengrad
    
    wout+= hlayer_act.T.dot(d_output) *lr
    bout+=np.sum(d_output,axis=0,keepdims=True) *lr
    wh+=X.T.dot(d_hiddenlayer) *lr
    
print("Input: \n "+str(X))
print("Actual output: \n",str(y))
print("Predicted output: \n",output)

Input: 
 [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual output: 
 [[0.92]
 [0.86]
 [0.89]]
Predicted output: 
 [[0.89396357]
 [0.88236091]
 [0.89321533]]


In [7]:
import pandas as pd
msg=pd.read_csv('bayes.csv',names=['message','label'])
print("dimension of dataset",msg.shape)
msg['labelnum']=msg.label.map({'pos':1,'neg':0})

X=msg.message
Y=msg.labelnum
print(X)
print(Y)

dimension of dataset (2, 2)
0    i love this sandwich
1    i hate this sandwich
Name: message, dtype: object
0    1
1    0
Name: labelnum, dtype: int64


In [8]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest=train_test_split(X,Y)
from sklearn.feature_extraction.text import CountVectorizer

count_vect=CountVectorizer()
xtrain_dtm=count_vect.fit_transform(xtrain)
xtest_dtm=count_vect.transform(xtest)

from sklearn.naive_bayes import MultinomialNB

elf=MultinomialNB().fit(xtrain_dtm,ytrain)
predicted=elf.predict(xtest_dtm)
from sklearn import metrics
print('Accuracy metrics')
print('Accuracy of the classifier is ',metrics.accuracy_score(ytest,predicted))
print('metrics confusion')
print(metrics.confusion_matrix(ytest,predicted))
print('recall and precision')
print(metrics.recall_score(ytest,predicted))
print(metrics.precision_score(ytest,predicted))


Accuracy metrics
Accuracy of the classifier is  0.0
metrics confusion
[[0 1]
 [0 0]]
recall and precision
0.0
0.0


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
